In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import ast
import json

def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host':'localhost', 'port':9200}])
    if _es.ping():
        print("Connected!")
    else:
        print("Could not connect!")
    return _es
def create_index(es_obj, index_name = 'recipes'):
    created = False
    setting = {
#         "settings":{
#             "number_of_shards":1,
#             "number_of_replicas":0
#         },
        "mappings":{
            "allrecipes":{
                "dynamic":"strict",
                "properties": {
                    "Title": {
                       "type":"text"
                    },
                    "Description": {
                        "type":"text"
                    },
                    "Ingredients": {
                        "type":"nested"
                    },
                    "Category": {
                        "type":"text"
                    },
                    "Avatar": {
                        "type":"text"
                    },
                    "Instructions": {
                        "type":"nested"
                    },
                    "PrepTime": {
                        "type":"text"
                    },
                    "CookTime": {
                        "type":"text"
                    },
                    "AdditionTime": {
                        "type":"text"
                    },
                    "TotalTime": {
                        "type":"text"
                    },
                    "Nutrition": {
                        "type":"text"
                    },
                    "Serving": {
                        "type":"integer"
                    },
                    "URL": {
                        "type":"text"
                    },
                    "Utensils": {
                        "type":"nested"
                    }
                }
            }
        }
    }
    try:
        if not es_obj.indices.exists(index_name):
            es_obj.indices.create(index=index_name, ignore=400, body=setting)
            print("Created Index!")
        created = True
    except:
        print("Could not create index!")
    finally:
        return created
def store_record(es_obj, index_name, type_name, record):
    try:
        outcome = es_obj.index(index=index_name, doc_type=type_name, body=record) 
    except Exception as ex:
        print('Error in indexing data')
        print(str(ex.info))

In [4]:
data = pd.read_csv('datasets/recipes/all_Recipes.csv')

In [5]:
data[:1]

,Title,Category,Avatar,Description,Ingredients,Instructions,PrepTime,CookTime,AdditionTime,TotalTime,Serving,Nutrition,Url,Utensils
0,Mexican-Inspired Watermelon Sorbet (Nieve de S...,Mexican,https://imagesvc.meredithcorp.io/v3/mm/image?u...,This watermelon sorbet can be made with or wit...,"['7\u2009½ cups seedless watermelon, chopped i...",['Place 2 1/2 cups watermelon into an electric...,10 mins,10 mins,4 hrs 20 mins,4 hrs 40 mins,10,127 calories; protein 0.1g; carbohydrates 32.9...,https://www.allrecipes.com/recipe/279207/,"['an electric blender', 'a saucepan', 'a close..."


In [17]:
# es_obj = connect_elasticsearch()
# create_index(es_obj)
# for index, row in data.iterrows():
#     record = {
#         "Title":        row["Title"] or "None",
#         "Description":  row["Description"] or "None",
#         "Ingredients":  ast.literal_eval(row["Ingredients"]) or "None",
#         "Category":     str(row["Category"]) or "None",
#         "Avatar":       str(row["Avatar"]) or "None",
#         "Instructions": ast.literal_eval(row["Instructions"]) or "None",
#         "PrepTime":     str(row["PrepTime"]) or "None",
#         "CookTime":     str(row["CookTime"]) or "None",
#         "AdditionTime": str(row["AdditionTime"]) or "None",
#         "TotalTime":    str(row["TotalTime"]) or "None",
#         "Nutrition":    str(row["Nutrition"]) or "None",
#         "Serving":      row["Serving"],
#         "URL":          row["Url"],
#         "Utensils":     ast.literal_eval(row["Utensils"]) or "None",
#     }
#     record = json.dumps(record)
#     store_record(es_obj,'recipes','allrecipes', record)

Connected!


In [16]:
def search(es_object, index_name, search):
    res = es_object.search(index=index_name, body=search)
    return res